In [26]:
# Import required libraries
from openai import OpenAI
import json
from healthcare_tools import tools_list, discharge_patient, get_patient_status

# Get user input
print("Enter your question or request:")
user_input = input().strip()
print("You entered: ", user_input)
print(user_input)

Enter your question or request:


 How is the patient 100001 doing?


You entered:  How is the patient 100001 doing?
How is the patient 100001 doing?


In [28]:

# Initialize OpenAI client
client = OpenAI()

# Set up the initial conversation messages
messages = [
    # System message defines the AI's role and available tools
    {"role": "system", "content": "You are a helpful assistant. Either answer a question or invoke the following tools:"\
          "discharge_patient(patient_id)"\
          "get_patient_status(patient_id)"
    },
    # Add the user's input to the conversation
    {"role": "user", "content": user_input}
]

# Make the initial API call to OpenAI
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools_list
)


In [41]:
try:
    # Check if the assistant wants to use any tools
    if response.choices[0].message.tool_calls:
        # Extract tool call information. 
        tool = response.choices[0].message.tool_calls[0]
        tool_name = tool.function.name
        tool_arguments = tool.function.arguments
        tool_id = tool.id

        print ("tool_name:", tool_name)
        print ("tool_arguments:", tool_arguments)
        print ("tool_id:", tool_id)
        # Add the assistant's response to the conversation history
        messages.append(response.choices[0].message)
        
        # Parse the tool arguments and execute the appropriate function
        data = json.loads(tool_arguments)
        response_message = ""
        if tool_name == 'discharge_patient':
            patient_id = data['patient_id']
            response_message = discharge_patient(patient_id)
        elif tool_name == 'get_patient_status':
            patient_id = data['patient_id']
            response_message = get_patient_status(patient_id)
    else:
        # If no tools were called, print the direct response
        print(response.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {str(e)}")
print ("response_message:", response_message)

tool_name: get_patient_status
tool_arguments: {"patient_id":"100001"}
tool_id: call_1mPJ00VhRL7UAPD391U3lPGe
response_message: Patient details for 100001


In [45]:
try:
# Add the tool's response to the conversation
        messages.append({
            "role": "tool",
            "tool_call_id": tool_id,
            "name": tool_name,
            "content": response_message,
        })

        print ("messages:", messages)
        # Make a second API call to get the assistant's final response
        second_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        print("Final response:", second_response.choices[0].message.content)
    

# Error handling
except Exception as e:
    print(f"An error occurred: {str(e)}")


messages: [{'role': 'system', 'content': 'You are a helpful assistant. Either answer a question or invoke the following tools:discharge_patient(patient_id)get_patient_status(patient_id)'}, {'role': 'user', 'content': 'How is the patient 100001 doing?'}, ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_1mPJ00VhRL7UAPD391U3lPGe', function=Function(arguments='{"patient_id":"100001"}', name='get_patient_status'), type='function')]), ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_1mPJ00VhRL7UAPD391U3lPGe', function=Function(arguments='{"patient_id":"100001"}', name='get_patient_status'), type='function')]), ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_1mPJ00VhRL7UAPD391U3lPGe', functi